# Face Detection with MediaPipe Tasks

This notebook shows you how to use the MediaPipe Tasks Python API to detect faces in images.

## Visualization utilities

To better demonstrate the Face Detector API, we have created a set of visualization tools that will be used in this colab. These will draw a bounding box around detected faces, as well as markers over certain detected points on the faces.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2


def draw_landmarks_on_image(rgb_image, detection_result):
    face_landmarks_list = detection_result.face_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected faces to visualize.
    for idx in range(len(face_landmarks_list)):
        face_landmarks = face_landmarks_list[idx]

        # Draw the face landmarks.
        face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        face_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
        ])

        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_tesselation_style())
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_contours_style())
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_iris_connections_style())

    return annotated_image


def plot_face_blendshapes_bar_graph(face_blendshapes):
    # Extract the face blendshapes category names and scores.
    face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
    face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
    # The blendshapes are ordered in decreasing score value.
    face_blendshapes_ranks = range(len(face_blendshapes_names))

    fig, ax = plt.subplots(figsize=(12, 12))
    bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
    ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
    ax.invert_yaxis()

    # Label each bar with values
    for score, patch in zip(face_blendshapes_scores, bar.patches):
        plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

    ax.set_xlabel('Score')
    ax.set_title("Face Blendshapes")
    plt.tight_layout()
    plt.show()

## Download test image

To demonstrate Face Detection, you can download a sample image using the following code. Credits: https://pixabay.com/photos/brother-sister-girl-family-boy-977170/

In [ ]:
!curl https://i.imgur.com/Vu2Nqwb.jpeg -s -o image.jpg

IMAGE_FILE = 'image.jpg'
# IMAGE_FILE = 'IMG_7676.jpeg'

import cv2

img = cv2.imread(IMAGE_FILE)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

Optionally, you can upload your own image from your computer. To do this, uncomment the following code cell.

## Running inference and visualizing the results

The final step is to run face detection on your selected image. This involves creating your FaceDetector object, loading your image, running detection, and finally, the optional step of displaying the image with visualizations.

You can check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/face_detector/python) to learn more about configuration options that this solution supports.

In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='../../data/mediapipe/face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
    num_faces=2)

detector = vision.FaceLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file(IMAGE_FILE)

# STEP 4: Detect face landmarks from the input image.
detection_result = detector.detect(image)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
plt.imshow(annotated_image)

In [ ]:
annotated_image.shape

In [ ]:
len(detection_result.face_landmarks[0])

In [ ]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Sample data: Replace this with your actual face landmarks data.
face_landmarks_3d = np.array([[landmark.x, landmark.y, landmark.z] for landmark in detection_result.face_landmarks[0]])

# Create the 3D scatter plot.
fig = go.Figure(data=[go.Scatter3d(
    x=face_landmarks_3d[:, 0],
    y=face_landmarks_3d[:, 1],
    z=face_landmarks_3d[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=face_landmarks_3d[:, 2],  # Color by Z axis value
        colorscale='Viridis',  # Color scale
        opacity=0.8
    )
)])

# Update the layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    width=700,
    height=700,
    margin=dict(r=20, l=10, b=10, t=10)
)

# Show plot
pio.show(fig)

In [ ]:
import cv2
import math
import numpy as np
from scipy.spatial import Delaunay

image = cv2.imread(IMAGE_FILE)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

landmarks_3d = np.array([[landmark.x, landmark.y, landmark.z] for landmark in detection_result.face_landmarks[0]])
landmarks_2d = landmarks_3d[:, :2]

print(f'min: {landmarks_3d[:, 0].min()} max: {landmarks_3d[:, 0].max()}')
print(f'min: {landmarks_3d[:, 1].min()} max: {landmarks_3d[:, 1].max()}')
print(f'min: {landmarks_3d[:, 2].min()} max: {landmarks_3d[:, 2].max()}')

k = 3

height, width = 36, 36

# Project 3D landmarks onto a cylinder
# u = np.arctan(landmarks_3d[:, 2] / landmarks_3d[:, 0])
u = landmarks_3d[:, 0]
v = landmarks_3d[:, 1]

print(f'u.min: {u.min()} u.max: {u.max()}')
print(f'v.min: {v.min()} v.max: {v.max()}')

# Normalize u and v coordinates to the output image size
# u = (u - u.min()) / (u.max() - u.min()) * width
# v = (v - v.min()) / (v.max() - v.min()) * height

print(f'u.min: {u.min()} u.max: {u.max()}')
print(f'v.min: {v.min()} v.max: {v.max()}')

In [ ]:
plt.scatter(u, v, c=landmarks_3d[:, 2], cmap='viridis')
# Make the coordinate system origin the top-left corner
plt.gca().invert_yaxis()
plt.show()

In [ ]:
triangulation = Delaunay(landmarks_2d)

# Fill the triangles with the average color of the triangle vertices
# normalized_image = np.zeros((height, width, 3), dtype=np.uint8)
normalized_image = np.zeros_like(image)

width = image.shape[1]
height = image.shape[0]

for triangle_indices in triangulation.simplices:
    print(f'triangle_indices: {triangle_indices}')
    # Get coordinates of the source triangle in the input image
    source_triangle = landmarks_2d[triangle_indices].astype(np.float32)
    print(f'source_triangle: {source_triangle}')
    
    source_triangle[:, 0] = source_triangle[:, 0] * width
    source_triangle[:, 1] = source_triangle[:, 1] * height
    
    # Convert the source_triangle to integer
    source_triangle = source_triangle.astype(np.int32)
    
    print(f'source_triangle: {source_triangle}')
    
    random_color = np.random.randint(0, 256, 3)
    r, g, b = float(random_color[0]), float(random_color[1]), float(random_color[2])
    print(f'random_color: {random_color}')
    
    # Draw the triangle on the normalized image
    cv2.fillConvexPoly(normalized_image, source_triangle, color=(r, g, b))
    
    # break

plt.imshow(normalized_image)